# Mongochain Demo: MongoDB Atlas as an Agent Memory Layer

This notebook demonstrates how to use **mongochain** to create AI agents with MongoDB Atlas-backed memory.

## Features Demonstrated
1. Creating an agent with a persona
2. Chatting with the agent (memories are stored automatically)
3. Changing the agent's persona dynamically
4. Multi-agent collaboration
5. Inspecting memories in MongoDB

## Setup

First, install the required dependencies:

In [ ]:
# Install dependencies (uncomment and run in Colab)
# !pip install pymongo voyageai openai anthropic google-generativeai
# !pip install git+https://github.com/robinvarghese/mongochain.git

In [ ]:
# Configure your API keys
# In Colab, you can use: from google.colab import userdata

MONGO_URI = "mongodb+srv://<username>:<password>@<cluster>.mongodb.net/"
VOYAGE_API_KEY = "voy-..."  # Get from https://www.voyageai.com/
LLM_API_KEY = "sk-..."  # OpenAI, Anthropic, or Google API key

# Uncomment for Colab secrets:
# from google.colab import userdata
# MONGO_URI = userdata.get('MONGO_URI')
# VOYAGE_API_KEY = userdata.get('VOYAGE_API_KEY')
# LLM_API_KEY = userdata.get('OPENAI_API_KEY')

## 1. Create Your First Agent

Let's create an agent named "alice" with a friendly research assistant persona.

In [ ]:
from mongochain import MongoAgent

# Create an agent with OpenAI (default provider)
alice = MongoAgent(
    name="alice",
    persona="You are a friendly research assistant who loves explaining complex topics in simple terms. You are enthusiastic and use analogies to help people understand.",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=LLM_API_KEY,
    llm_provider="openai"  # Can also be "anthropic" or "google"
)

**Check MongoDB Atlas!** You should now see a database named `alice` with three collections:
- `short_term_memory` - Recent context (auto-expires)
- `long_term_memory` - Persistent facts (vector search enabled)
- `conversation_history` - Full chat log

## 2. Chat with the Agent

Now let's have a conversation. Each message is automatically stored in MongoDB.

In [ ]:
# First message
response = alice.chat("Hello! Can you explain what vector databases are?")
print("Alice:", response)

In [ ]:
# Follow-up - Alice should remember the context
response = alice.chat("How does MongoDB Atlas support vector search?")
print("Alice:", response)

In [ ]:
# Check memory stats
stats = alice.get_memory_stats()
print("Memory Statistics:")
print(f"  Short-term memories: {stats['short_term_count']}")
print(f"  Long-term memories: {stats['long_term_count']}")
print(f"  Conversation messages: {stats['conversation_count']}")

## 3. Change the Agent's Persona

One powerful feature is the ability to change the agent's personality on the fly. Watch how the responses change!

In [ ]:
# Ask a question with the current friendly persona
response = alice.chat("What's the difference between SQL and NoSQL databases?")
print("Friendly Alice:", response)

In [ ]:
# Change to a grumpy professor persona
alice.set_persona(
    "You are a grumpy but brilliant computer science professor. "
    "You give very brief, slightly impatient answers but they're always accurate. "
    "You occasionally sigh or make sarcastic comments about obvious questions."
)

In [ ]:
# Ask the same question again - notice the different tone!
response = alice.chat("What's the difference between SQL and NoSQL databases?")
print("Grumpy Professor Alice:", response)

In [ ]:
# Try another persona - a pirate!
alice.set_persona(
    "You are a pirate who somehow became a database expert. "
    "You explain everything using nautical metaphors and pirate speak. "
    "Arrr!"
)

response = alice.chat("Tell me about database indexing.")
print("Pirate Alice:", response)

## 4. Multi-Agent Collaboration

Now let's create a second agent that can access Alice's memories. This demonstrates how agents can share knowledge.

In [ ]:
# Reset Alice to a normal persona first
alice.set_persona(
    "You are a research assistant specializing in databases and AI."
)

# Teach Alice something specific
response = alice.chat(
    "Let me tell you about our company's database setup: "
    "We use MongoDB Atlas with 3 shards, each with 64GB RAM. "
    "Our main collection has 50 million documents with vector embeddings for semantic search."
)
print("Alice learned:", response)

In [ ]:
# Manually store important information in Alice's long-term memory
alice.store_memory(
    "Company database specs: MongoDB Atlas, 3 shards, 64GB RAM each, "
    "50 million documents with vector embeddings.",
    memory_type="long_term"
)

In [ ]:
# Create Bob - a new agent who can read Alice's memories
bob = MongoAgent(
    name="bob",
    persona="You are a DevOps engineer who focuses on infrastructure and optimization.",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=LLM_API_KEY,
    collaborators=["alice"]  # Bob can read Alice's long-term memories!
)

In [ ]:
# Ask Bob about something Alice knows
response = bob.chat(
    "What do you know about our company's database setup? "
    "I heard Alice has information about it."
)
print("Bob (with Alice's knowledge):", response)

## 5. Using Different LLM Providers

Mongochain supports multiple LLM providers. Here's how to create agents with different backends:

In [ ]:
# Example with Anthropic Claude (uncomment to use)
# ANTHROPIC_API_KEY = "sk-ant-..."
# 
# claude_agent = MongoAgent(
#     name="claude_assistant",
#     persona="You are a helpful coding assistant.",
#     mongo_uri=MONGO_URI,
#     voyage_api_key=VOYAGE_API_KEY,
#     llm_api_key=ANTHROPIC_API_KEY,
#     llm_provider="anthropic",
#     llm_model="claude-3-haiku-20240307"  # Optional: specify model
# )

In [ ]:
# Example with Google Gemini (uncomment to use)
# GOOGLE_API_KEY = "AIza..."
# 
# gemini_agent = MongoAgent(
#     name="gemini_assistant",
#     persona="You are a creative writing assistant.",
#     mongo_uri=MONGO_URI,
#     voyage_api_key=VOYAGE_API_KEY,
#     llm_api_key=GOOGLE_API_KEY,
#     llm_provider="google",
#     llm_model="gemini-1.5-pro"  # Optional: specify model
# )

## 6. Inspect Memories in MongoDB

You can directly query MongoDB to see the stored memories:

In [ ]:
from pymongo import MongoClient

# Connect directly to MongoDB
client = MongoClient(MONGO_URI)

# List all agent databases
print("Databases (agent memories):")
for db_name in client.list_database_names():
    if db_name not in ['admin', 'local', 'config']:
        print(f"  - {db_name}")

In [ ]:
# Inspect Alice's conversation history
alice_db = client['alice']

print("Alice's Conversation History:")
print("=" * 50)
for msg in alice_db.conversation_history.find().sort('timestamp', -1).limit(5):
    print(f"[{msg['role']}]: {msg['content'][:100]}...")
    print()

In [ ]:
# Inspect long-term memories (with embeddings)
print("Alice's Long-Term Memories:")
print("=" * 50)
for mem in alice_db.long_term_memory.find({}, {'embedding': 0}).limit(5):
    print(f"Content: {mem['content'][:100]}...")
    print(f"Timestamp: {mem['timestamp']}")
    print()

## 7. Cleanup

Optionally clear memories when done testing:

In [ ]:
# Clear specific memory types
# alice.clear_memories("conversation")  # Clear only conversation history
# alice.clear_memories("short_term")    # Clear only short-term memories

# Or clear everything (careful!)
# alice.clear_memories()  # Clears all memories

## Summary

In this demo, we've shown how **mongochain** uses MongoDB Atlas as a powerful memory layer for AI agents:

1. **Easy Agent Creation** - Just provide a name, persona, and API keys
2. **Automatic Memory Management** - Conversations are stored automatically in MongoDB
3. **Multiple Memory Types**:
   - Short-term (TTL auto-expiry)
   - Long-term (vector search for semantic retrieval)
   - Conversation history
4. **Dynamic Personas** - Change agent personality on the fly
5. **Multi-Agent Collaboration** - Agents can share memories
6. **Multi-Provider LLM Support** - OpenAI, Anthropic, or Google

Check your MongoDB Atlas cluster to see the agent databases and explore the stored memories!